In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
%cd drive/Shareddrives/CS263\ Project

Mounted at /content/drive
/content/drive/Shareddrives/CS263 Project


In [ ]:
#Importing necessary libraries 
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tokenize
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing import sequence
from keras.layers import SimpleRNN, Activation, Dense, Dropout, Input, Embedding, LSTM
from keras.models import Model
from keras.metrics import Precision
from keras.utils import pad_sequences

In [ ]:
#Inputand read in spam dataset, check head of dataset
df = pd.read_csv('spam.csv',delimiter=',',encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
#Create input vectors and match to correct column
X_input = df.v2
Y_input = df.v1

#Use label encoder to encode target labels (not input values)
labels = preprocessing.LabelEncoder()

#Reshape Y 
Y = labels.fit_transform(Y_input)
Y = Y.reshape(-1,1)

In [ ]:
#Split dataset into training and testing data
X_train, X_test, Y_train, Y_test = train_test_split(X_input,Y,test_size=0.2)

In [ ]:
#Tokenize columns
tok = Tokenizer(num_words=300)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=300)

In [ ]:
#Create a simple RNN, add different layers to the model
inputs = Input(name='inputs',shape=[300])
layer = Embedding(300,50,input_length=300)(inputs)
layer = SimpleRNN(256)(layer)
layer = Dense(256,name='FC1',activation='relu')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)

In [25]:
#Summarize and then compile model
model.summary()
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy', Precision()])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 300)]             0         
                                                                 
 embedding (Embedding)       (None, 300, 50)           15000     
                                                                 
 simple_rnn (SimpleRNN)      (None, 256)               78592     
                                                                 
 FC1 (Dense)                 (None, 256)               65792     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [26]:
#Fit model on training data 
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10, validation_split=0.2)

Epoch 1/10
28/28 [==============================] - 24s 826ms/step - loss: 0.1501 - accuracy: 0.9534 - precision_3: 0.8581 - val_loss: 0.0576 - val_accuracy: 0.9843 - val_precision_3: 0.9681
Epoch 2/10
28/28 [==============================] - 25s 895ms/step - loss: 0.0699 - accuracy: 0.9787 - precision_3: 0.9379 - val_loss: 0.0747 - val_accuracy: 0.9809 - val_precision_3: 1.0000
Epoch 3/10
28/28 [==============================] - 27s 991ms/step - loss: 0.0658 - accuracy: 0.9806 - precision_3: 0.9594 - val_loss: 0.0554 - val_accuracy: 0.9843 - val_precision_3: 0.9400
Epoch 4/10
28/28 [==============================] - 23s 831ms/step - loss: 0.0641 - accuracy: 0.9798 - precision_3: 0.9514 - val_loss: 0.1209 - val_accuracy: 0.9731 - val_precision_3: 1.0000
Epoch 5/10
28/28 [==============================] - 24s 855ms/step - loss: 0.0684 - accuracy: 0.9795 - precision_3: 0.9365 - val_loss: 0.0532 - val_accuracy: 0.9832 - val_precision_3: 0.9780
Epoch 6/10
28/28 [===========================

In [ ]:
#check accuracy and precision on test data
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=300)

In [ ]:
#Report final test accuracy 
#Acuracy: 94%
#Precision: 72% 
final_test_accuracy = model.evaluate(test_sequences_matrix,Y_test)

35/35 [==============================] - 3s 96ms/step - loss: 0.1359 - accuracy: 0.9462 - precision: 0.7216
